# Get Data from each AMC

In [ ]:
import requests
import json
import pandas as pd
import os
import dotenv
import time
import base64
dotenv.load_dotenv()
FDAPI = os.environ["FUND_DAILY_API_KEY"]
FFAPI = os.environ["FUND_FACT_API_KEY"]
FundDailyKey = {
    'Ocp-Apim-Subscription-Key': FDAPI
}
FundFactKey = {
    'Ocp-Apim-Subscription-Key': FFAPI
}
project_type_keys = ['proj_retail_type', 'proj_term_flag', 'proj_term_dd', 'proj_term_mm','proj_term_yy']
specification_keys = ['spec_code', 'spec_desc']
suitability_keys = ['risk_spectrum', 'risk_spectrum_desc','fund_suitable_desc']

In [ ]:
def FundFactAPIGet(url, proj_unique_id, type, key):
    if type == 'amc':
        response = requests.get(url,headers=key)
        if response.status_code == 200:
            return json.loads(response.text)
    elif type in ['project_type', 'specification', 'suitability']:
        response = requests.get(url + proj_unique_id + '/' + type, headers=key)
        if response.status_code == 200:
            return {'json': json.loads(response.text), 'status_code': response.status_code}
        else:
            dummy_json = {key: None for key in project_type_keys} if type == 'project_type' else ({key: None for key in specification_keys} if type == 'specification' else {key: None for key in suitability_keys})
            return {'json': dummy_json, 'status_code': response.status_code}

In [ ]:
response = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/amc',None, 'amc', FundFactKey)
all_amc = pd.DataFrame(json.loads(response.text))

In [ ]:
for index_all, row_all in all_amc.iterrows():
    response = requests.get(f'https://api.sec.or.th/FundFactsheet/fund/amc/{row_all["unique_id"]}', headers=FundFactKey)
    amc_proj = pd.DataFrame(json.loads(response.text))
    amc_proj = amc_proj.drop(columns=['last_upd_date'])
    amc_proj = amc_proj[amc_proj['fund_status'] == 'RG']
    amc_proj = amc_proj.reset_index(drop=True)
    if not len(amc_proj.index) == 0:
        for index_amc_proj, row_amc_proj in amc_proj.iterrows():
                try:
                    response = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/',str(row_amc_proj['proj_id']),'project_type',FundFactKey)
                    amc_proj.at[index_amc_proj, 'proj_retail_type'] = response['json']['proj_retail_type']
                    amc_proj.at[index_amc_proj, 'proj_term_flag'] = response['json']['proj_term_flag']
                    amc_proj.at[index_amc_proj, 'proj_term_dd'] = response['json']['proj_term_dd']
                    amc_proj.at[index_amc_proj, 'proj_term_mm'] = response['json']['proj_term_mm']
                    amc_proj.at[index_amc_proj, 'proj_term_yy'] = response['json']['proj_term_yy']
                except Exception as e:
                    pass
                try:
                    response = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/',str(row_amc_proj['proj_id']),'specification',FundFactKey)
                    try:
                        amc_proj.at[index_amc_proj, 'spec_code'] =  ','.join(sub_res['spec_code'] for sub_res in response['json'])
                    except Exception as e:
                        pass
                    try:
                        amc_proj.at[index_amc_proj, 'spec_desc'] =  ','.join(sub_res['spec_desc'] for sub_res in response['json']) 
                    except Exception as e:
                        pass
                except Exception as e:
                    pass
                try:
                    response = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/',str(row_amc_proj['proj_id']),'suitability',FundFactKey)
                    amc_proj.at[index_amc_proj, 'risk_spectrum'] = response['json']['risk_spectrum']
                    try:
                        amc_proj.at[index_amc_proj, 'risk_spectrum_desc'] = base64.b64decode(response['json']['risk_spectrum_desc']).decode('utf-8')
                    except Exception as e:
                        amc_proj.at[index_amc_proj, 'risk_spectrum_desc'] = None
                    try:
                        amc_proj.at[index_amc_proj, 'fund_suitable_desc'] = base64.b64decode(response['json']['fund_suitable_desc']).decode('utf-8')
                    except Exception as e:
                        amc_proj.at[index_amc_proj, 'fund_suitable_desc'] = None
                except Exception as e:
                    pass
                time.sleep(0.1)
        amc_name = row_all['name_en'].split()
        os.mkdir(f'../Result_amc/{amc_name[0]}')
        amc_proj.to_csv(f'../Result_amc/{amc_name[0]}/{amc_name[0]}_funds.csv', encoding='utf8')